# iTorch / Torch7 implementation


**Gm = d** in lua/torch for LuaJIT. 


## Prerequisites

You need to install `torch` and (if you want to run this notebook) `iTorch`. Works for me on Ubuntu 16.04 and MacOS 10.11. Not gonna say the Mac installation is easy though. Everything was very smooth on Ubuntu.

- [Getting started with Torch](http://torch.ch/docs/getting-started.html)
- [iTorch Requirements](https://github.com/facebook/iTorch#requirements)
- [Torch cheatsheet](https://github.com/torch/torch7/wiki/Cheatsheet)


## Demo

We'll start with the example from [the torch7 documentation](http://torch7.readthedocs.io/en/rtd/maths/#eigenvalues-svd-linear-system-solution). 

In [1]:
G = torch.Tensor({{6.80, -2.11,  5.66,  5.97,  8.23},
                  {-6.05, -3.30,  5.36, -4.44,  1.08},
                  {-0.45,  2.58, -2.70,  0.27,  9.04},
                  {8.32,  2.71,  4.35,  -7.17,  2.14},
                  {-9.67, -5.14, -7.26,  6.08, -6.87}}):t()  -- transpose

In [2]:
d = torch.Tensor({{4.02,  6.19, -8.22, -7.57, -3.03},
                  {-1.56,  4.00, -8.67,  1.75,  2.86},
                  {9.81, -4.09, -4.57, -8.61,  8.99}}):t()

Later we can use lower-level linear algebraic functions, but for now here's the solution using LAPACK's `gesv`, which [is described](http://www.crest.iu.edu/research/mtl/reference/html/gesvLapackMatAVectorIntLapackMatB.html) like this:

> Computes the solution to a real system of linear equations `A*X=B` (simple driver). LU decomposition with partial pivoting and row interchanges is used to solve the system.

In [3]:
m_est = torch.gesv(d, G)

In [4]:
d:dist(G*m_est)

1.1808097433449e-14	


## Mauricio's example

In [5]:
-- Define model
M = 50
m = torch.zeros(M, 1)
m[{{11,15}, {}}] = 1.0
m[{{16,27}, {}}] = -0.3
m[{{28,35}, {}}] = 2.1

In [6]:
Plot = require 'itorch.Plot'

In [7]:
x = torch.range(1,m:size(1))
plot = Plot():line(x, m[{{},1}]):circle(x, m[{{},1}]):title('Model m'):draw()

In [8]:
-- Discrete kernel G
N = 20
L = 100
alpha = 0.8

In [9]:
M = 50
x = torch.range(0, M, 1) * L/(M-1)
dx = L/(M-1)
r = torch.range(0, N, 1) * L/(N-1)

In [10]:
G = torch.zeros(N, M)

In [11]:
for j = 1,M do
    for k = 1,N do
        t = torch.abs(r[{k}] - x[{j}])
        G[{{k},{j}}] = dx * torch.exp(-alpha * torch.pow(t, 2))
    end
end

In [12]:
d = G * m

In [13]:
Plot = require 'itorch.Plot'
x = torch.range(1,d:size(1))
plot = Plot():line(x, d[{{},1}]):circle(x, d[{{},1}]):title('Data d'):draw()

## Noise free: minimum norm

In [14]:
-- Compute data
d = G * m

In [15]:
-- Minimum norm solution
m_est = G:t() * torch.inverse(G * G:t()) * d
d_pred = G * m_est

In [16]:
-- Plot as before
Plot = require 'itorch.Plot'
x = torch.range(1,d_pred:size(1))
plot = Plot():line(x, d_pred[{{},1}]):circle(x, d_pred[{{},1}]):title('Predicted data d_pred'):draw()

## Least squares solution using GESV

This uses factorization instead of finding the inverse...

In [17]:
-- Use torch.gels()
m_est = torch.gels(d, G)
d_pred = G * m_est

-- Plot
Plot = require 'itorch.Plot'
x = torch.range(1,d_pred:size(1))
plot = Plot():line(x, d[{{},1}], 'blue'):circle(x, d[{{},1}], 'blue')
             :line(x, d_pred[{{},1}]):circle(x, d_pred[{{},1}]):title('blue: d — red: d_pred')
             :draw()

In [18]:
d:dist(G * m_est)

1.566162308831e-15	


## With noise: damped least squares

In [19]:
-- Add noise
dc = G * m
s = 1
d = dc + s * torch.DoubleTensor(d:size()):uniform(0,1)

In [20]:
-- Compute using the second form
I = torch.eye(N)
mu = 2.5
m_est = G:t() * torch.inverse(G*G:t() + mu*I) * d
d_pred = G * m_est

In [21]:
-- Plot
Plot = require 'itorch.Plot'
x = torch.range(1,d_pred:size(1))
plot = Plot():line(x, d[{{},1}], 'blue'):circle(x, d[{{},1}], 'blue')
             :line(x, d_pred[{{},1}]):circle(x, d_pred[{{},1}]):title('blue: d — red: d_pred')
             :draw()

## Interlude: convmtx

We need to implement convmtx, the matrix equivalent of the convolutional operator. This helps us make the first derivative.

There's no Toeplitz helper function that I can find, so we'll have to do it a slightly different way from the one we used in the [NumPy](NumPy.ipynb) notebook.

In [22]:
function convmtx (h, n)
    local m = h:size()[1] - 1
    local c = torch.Tensor(n, n+m):fill(0)
    for i = 1, n do
        c[{ i,{i,i+m} }] = h
    end
    return c
end

In [23]:
h = torch.Tensor({1,-1})
n = 8  -- number of rows in conv. matrix

In [24]:
convmtx(h, n)

 1 -1  0  0  0  0  0  0  0
 0  1 -1  0  0  0  0  0  0
 0  0  1 -1  0  0  0  0  0
 0  0  0  1 -1  0  0  0  0
 0  0  0  0  1 -1  0  0  0
 0  0  0  0  0  1 -1  0  0
 0  0  0  0  0  0  1 -1  0
 0  0  0  0  0  0  0  1 -1
[torch.DoubleTensor of size 8x9]



As far as I can tell, there's no simple way to index off the last column, as with the `-1` trick in Python. But we know the number of columns is `8 + h.size()[1] - 1` so we can just substract one from that...

In [25]:
cols = n + h:size()[1] - 1
print(convmtx(h, n)[{ {},{1, cols-1} }])

 1 -1  0  0  0  0  0  0
 0  1 -1  0  0  0  0  0
 0  0  1 -1  0  0  0  0
 0  0  0  1 -1  0  0  0
 0  0  0  0  1 -1  0  0
 0  0  0  0  0  1 -1  0
 0  0  0  0  0  0  1 -1
 0  0  0  0  0  0  0  1
[torch.DoubleTensor of size 8x8]



## With noise: damped LS with 1st derivative regularization

In [26]:
cols = M + h:size()[1] - 1
W = convmtx(h, M)[{ {},{1, cols-1} }]

Now we solve:

$$ \hat{\mathbf{m}} = (\mathbf{G}^\mathrm{T} \mathbf{G} + \mu \mathbf{W}^\mathrm{T} \mathbf{W})^{-1} \mathbf{G}^\mathrm{T} \mathbf{d} \ \ $$

In [27]:
m_est = torch.inverse(G:t()*G + mu*W:t()*W) * G:t() * d
d_pred = G * m_est

In [28]:
-- Plot
Plot = require 'itorch.Plot'
x = torch.range(1,d_pred:size(1))
plot = Plot():line(x, d[{{},1}], 'blue'):circle(x, d[{{},1}], 'blue')
             :line(x, d_pred[{{},1}]):circle(x, d_pred[{{},1}]):title('blue: d — red: d_pred')
             :draw()

In [29]:
-- Plot
Plot = require 'itorch.Plot'
x = torch.range(1,m_est:size(1))
plot = Plot():line(x, m[{{},1}], 'blue', 'blue')
             :line(x, m_est[{{},1}]):title('blue: m — red: m_est')
             :draw()

## Plotting demo

From [the iTorch readme](https://github.com/facebook/iTorch/blob/master/README.md).

In [30]:
x1 = torch.randn(40):mul(100)
y1 = torch.randn(40):mul(100)
x2 = torch.randn(40):mul(100)
y2 = torch.randn(40):mul(100)
x3 = torch.randn(40):mul(200)
y3 = torch.randn(40):mul(200)


-- scatter plots
plot = Plot():circle(x1, y1, 'red', 'hi'):circle(x2, y2, 'blue', 'bye'):draw()
plot:circle(x3,y3,'green', 'yolo'):redraw()
plot:title('Scatter Plot Demo'):redraw()
plot:xaxis('length'):yaxis('width'):redraw()
plot:legend(true)
plot:redraw()

## BLAS and CUDA examples

From [Nicholas Leonard's examples](https://github.com/nicholas-leonard/slides/blob/master/torch7.md).

In [31]:
batchSize, inputSize, outputSize = 4, 2, 3
input = torch.FloatTensor(batchSize, inputSize):uniform(0,1)
weight = torch.FloatTensor(outputSize, inputSize):uniform(0,1)
output = torch.FloatTensor(4, 3)  -- Must give size here, AFAICT

In [32]:
output:addmm(0, output, 1, input, weight:t())  -- transpose of weight

In [33]:
output

 0.5642  0.2668  0.5441
 0.5796  0.1830  0.5734
 0.3035  0.2187  0.2808
 1.1134  0.3657  1.0992
[torch.FloatTensor of size 4x3]



How about `CudaTensor`s?

Note that you need an nVidia graphics card with a recent version of CUDA installed, and you will need to install `cutorch` with `luarocks`.

In [34]:
require 'cutorch'
input = torch.CudaTensor(batchSize, inputSize):uniform(0,1)
weight = torch.CudaTensor(outputSize, inputSize):uniform(0,1)
output = torch.CudaTensor(4,3)

output:addmm(0, output, 1, input, weight:t())

In [35]:
output

 0.1897  0.4827  0.4121
 0.2829  0.9010  0.5999
 0.3995  0.8799  0.8786
 0.2224  0.7813  0.4658
[torch.CudaTensor of size 4x3]



Seems to work, not sure how to time these things... 